In [1]:
from PIL import Image
import torch
import torch.nn as nn
from transformers import CLIPProcessor, CLIPModel
import onnxruntime

In [2]:
image = Image.open("./CLIP.png")
processor = CLIPProcessor.from_pretrained("./models/openai/clip-vit-base-patch32")
pixel_values = processor(images=image, return_tensors="pt").pixel_values


In [3]:
pixel_values.size()

torch.Size([1, 3, 224, 224])

In [7]:
# 测试onnxruntime
onnx_session = onnxruntime.InferenceSession("image_onnx_with_text.onnx")
prob = onnx_session.run(["logit"], {"pixel_values": pixel_values.numpy()})

In [8]:
prob_ = torch.tensor(prob)
prob_

tensor([[[25.0325, 19.4092, 18.5067]]])

In [9]:
torch.softmax(prob_,dim=-1)

tensor([[[0.9949, 0.0036, 0.0015]]])

## 直接计算结果

In [11]:
texts = ["a diagram", "a dog", "a cat"]
model = CLIPModel.from_pretrained("./models/openai/clip-vit-base-patch32")
input_data = processor(text=texts, images=image, return_tensors="pt")
output = model(**input_data).logits_per_image


In [12]:
print(output)

tensor([[25.0325, 19.4092, 18.5067]], grad_fn=<TBackward0>)
